In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = ''

In [3]:
df = pd.read_csv(path + '/data/data_pre.csv', encoding = 'cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  2248 non-null   int64 
 1   사건명     2248 non-null   object
 2   사건번호    2248 non-null   object
 3   선고일자    2248 non-null   object
 4   법원명     2248 non-null   object
 5   사건종류명   2248 non-null   object
 6   사건종류코드  2248 non-null   int64 
 7   판결유형    2248 non-null   object
 8   선고      2248 non-null   object
 9   판례상세링크  2248 non-null   object
 10  판시사항    2248 non-null   object
 11  판결요지    2248 non-null   object
 12  참조조문    2248 non-null   object
 13  참조판례    2246 non-null   object
 14  판례내용    2248 non-null   object
 15  X       2248 non-null   object
dtypes: int64(2), object(14)
memory usage: 281.1+ KB


# 1. 판례내용 임베딩

In [4]:
from nltk import sent_tokenize

## 1.1. FastText 학습

In [5]:
word_tokenize = []
with open(path + '/data/kkma_word.txt') as file :
    for i in file :
        word_tokenize.append(i.strip())
print(word_tokenize[0:10])

['상고', '이유', '판단', '항소심', '피고인', '출석', '개정', '원칙', '이지만', '형사']


In [6]:
from gensim.models import FastText

In [7]:
model = FastText(word_tokenize, vector_size = 512, window = 7, min_count = 5, workers = 4)

In [8]:
model.save(path + '/model/fasttext/fasttext_kkma.model')

## 1.2. 문장 토큰화

##### sent_tokenize 사용

In [9]:
from tqdm import tqdm

In [10]:
sentence_tokened = []
for x in tqdm(df['X']):
    sentence_tokened.append(sent_tokenize(x))

100%|████████████████████████████████████████████████████████████████████████████| 2248/2248 [00:01<00:00, 1141.09it/s]


In [11]:
len(sentence_tokened)

2248

In [12]:
sentence_tokened[0]

['  상고이유를 판단한다.항소심에서도 피고인의 출석 없이 개정하지 못하는 것이 원칙이지만형사소송법   피고인이 항소심 공판기일에 출정하지 않아 기일을 정하였는데도 정당한 없이 그 기일에도 출정하지 않은 때에는 피고인의 진술 없이 판결할 수 있다형사소송법 .',
 '이와 같이 피고인이 불출석한 상태에서 그 진술 없이 판결하기 위해서는 피고인이 적법한 공판기일 통지를 받고서도 회 연속으로 정당한 이유 없이 출정하지 않은 경우에 해당하여야 한다  참조.',
 '이때 적법한 공판기일 통지란 소환장의 송달형사소송법  및 소환장 송달의 의제형사소송법 의 경우에 한정되는 것이 아니라 적어도 피고인의 이름죄명출석 일시출석 장소가 명시된 공판기일 변경명령을 송달받은 경우형사소송법 도 포함된다.',
 '기록에 따르면  원심은  피고인에 대하여 적법한 공시송달 결정을 하였고 같은 날 공판기일 변경명령도 하여 공시송달의 방법으로  이를 송달한  위 공판기일 변경명령에는 피고인의 이름죄명은 물론 제회 공판기일에 관한 일시장소까지 명시된  원심은  제회 공판기일에 피고인이 불출석하자 공판기일을 로 연기한 후 그 소환장을 공시송달의 방법으로  송달한  원심은  제회 공판기일에 피고인이 불출석한 상태에서 공판절차를 진행하여 변론을 종결한 후 선고기일 소환장을 공시송달의 방법으로  송달한  원심판결을 선고한을 알 수 있다.',
 '이와 같은관계를 앞서 본 법리에 비추어 살펴보면 원심이 제회 공판기일에 관한 공판기일 변경명령 및 제회 공판기일에 관한 소환장을 적법한 공판기일 통지로 보고 진행한 소송절차에 항소심의 불출석 재판에 관한 법령을 위반한 잘못이 없다.',
 '그러므로 상고를 기각하기로 하여 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.대법관 민유숙재판장 조재연 이동원 천대엽주심']

## 1.3. 문장 임베딩

In [13]:
model = FastText.load(path + '/model/fasttext/fasttext_kkma.model')

In [14]:
embeddings = []

for sentences in tqdm(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = model.wv.get_sentence_vector(sentence)
        else:
            embedding += model.wv.get_sentence_vector(sentence)
        count += 1
    embeddings.append(embedding / count)

100%|██████████████████████████████████████████████████████████████████████████████| 2248/2248 [02:37<00:00, 14.30it/s]


## 1.4. 임베딩 값 저장

In [15]:
import pickle as pk

In [16]:
with open(path + '/data/fasttext_embedding.pickle', 'wb') as fw:
    pk.dump(embeddings, fw)